# GA Data Science 10 (DAT10) - Lab 14

### Recommendation Systems

Francesco Mosconi, Justin Breucop

### Today

1. Simple similarity based recommendation system
2. Recsys

## Similarity based Recommendation System: Beers


Let's build a recommendation system to recommend types of beers based on user reviews

Usual imports (numpy, pandas)

In [25]:
import pandas as pd
import numpy as np

First of all let's get the data

In [26]:
! curl -O https://s3.amazonaws.com/demo-datasets/beer_reviews.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27.3M  100 27.3M    0     0   425k      0  0:01:05  0:01:05 --:--:--  389k


Import data in a pandas dataframe called "allbeers". Use the compression keyword

In [27]:
allbeers = pd.read_csv("beer_reviews.tar.gz", compression='gzip')

/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


Let's look at the data

In [28]:
allbeers.head()

,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


Let's restrict this to the top 250 beers. Use the value_counts() method select the top 250 beers.
Assign the selected beers to a dataset called df

In [29]:
n = 250
top_n = allbeers.beer_name.value_counts().index[:n]
df = allbeers[allbeers.beer_name.isin(top_n)]
df.head()

,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
798,1075,Caldera Brewing Company,1212201268,4.5,4.5,4,grumpy,American Double / Imperial Stout,4.0,4.5,Imperial Stout,NaN,42964
1559,11715,Destiny Brewing Company,1137124057,4.0,3.5,4,blitheringidiot,American Pale Ale (APA),3.5,3.5,Pale Ale,4.5,26420
1560,11715,Destiny Brewing Company,1129504403,4.0,2.5,4,NeroFiddled,American Pale Ale (APA),4.0,3.5,Pale Ale,4.5,26420
1563,11715,Destiny Brewing Company,1137125989,3.5,3.0,4,blitheringidiot,American IPA,4.0,4.0,IPA,NaN,26132
1564,11715,Destiny Brewing Company,1130936611,3.0,3.0,3,Gavage,American IPA,4.0,3.5,IPA,NaN,26132


How big is this dataset?

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355275 entries, 798 to 1586564
Data columns (total 13 columns):
beer_reviews/         355275 non-null object
brewery_name          355275 non-null object
review_time           355275 non-null float64
review_overall        355275 non-null float64
review_aroma          355275 non-null float64
review_appearance     355275 non-null float64
review_profilename    355175 non-null object
beer_style            355275 non-null object
review_palate         355275 non-null float64
review_taste          355275 non-null float64
beer_name             355275 non-null object
beer_abv              353477 non-null float64
beer_beerid           355275 non-null float64
dtypes: float64(8), object(5)
memory usage: 37.9+ MB


### Pivot Table

Aggregate the data in a pivot table using the pivot_table method. Display the mean review_overall for each beer_name aggregating the review_overall values by review_profilename. Use the mean as aggregator.

In [31]:
df_pivot = pd.pivot_table(df, values=["review_overall"],
        columns=["beer_name", "review_profilename"],
        aggfunc=np.mean)
#pivot_table converts to a multi-index series. Unstack converts to a dataframe where the last index becomes our column head
df_wide = df_pivot.unstack(-1)
df_wide

review_profilename                                                 0110x011  \
               beer_name                                                      
review_overall #9                                                       NaN   
               120 Minute IPA                                           NaN   
               1554 Enlightened Black Ale                               NaN   
               60 Minute IPA                                            NaN   
               90 Minute IPA                                            5.0   
               Aecht Schlenkerla Rauchbier Märzen                       NaN   
               AleSmith IPA                                             5.0   
               AleSmith Speedway Stout                                  4.5   
               Allagash White                                           NaN   
               Alpha King Pale Ale                                      5.0   
               Anchor Liberty Ale                                       NaN   
               Anchor Old Foghorn                                       NaN   
               Anchor Porter                                            NaN   
               Anchor Steam Beer                                        NaN   
               ApriHop                                                  NaN   
               Arrogant Bastard Ale                                     NaN   
               Ayinger Celebrator Doppelbock                            NaN   
               Bass Pale Ale                                            NaN   
               Bell's Expedition Stout                                  4.0   
               Bell's Hopslam Ale                                       4.5   
               Bell's Java Stout                                        NaN   
               Bell's Kalamazoo Stout                                   NaN   
               Bell's Oberon Ale                                        NaN   
               Bell's Special Double Cream Stout                        NaN   
               Big Bear Black Stout                                     NaN   
               Black Butte Porter                                       NaN   
               Blanche De Chambly                                       NaN   
               Blind Pig IPA                                            4.5   
               Blue Moon Belgian White                                  NaN   
               Boddingtons Pub Ale                                      NaN   
...                                                                     ...   
               Stone Smoked Porter                                      NaN   
               Stone Sublimely Self-Righteous Ale                       5.0   
               Storm King Stout                                         4.0   
               Supplication                                             4.0   
               Ten FIDY                                                 4.0   
               Terrapin Coffee Oatmeal Imperial Stout                   NaN   
               The Abyss                                                4.5   
               Three Philosophers Belgian Style Blend (Quadrupel)       NaN   
               Titan IPA                                                NaN   
               Trappist Westvleteren 12                                 4.5   
               Trappistes Rochefort 10                                  NaN   
               Trappistes Rochefort 8                                   NaN   
               Tripel Karmeliet                                         NaN   
               Troegenator Double Bock                                  NaN   
               Trois Pistoles                                           NaN   
               Tröegs Hopback Amber Ale                                 NaN   
               Tröegs Mad Elf                                           4.5   
               Tröegs Nugget Nectar               

In [32]:
df_pivot

                beer_name         review_profilename
review_overall  #9                2xHops                3.0
                                  3Vandoo               4.0
                                  57andcounting         4.0
                                  AChamberlain          4.0
                                  ADR                   4.0
                                  AMoceri7              4.5
                                  AaronHomoya           3.5
                                  AaronRed              3.0
                                  AbePhroman            4.0
                                  Absumaster            3.5
                                  AdamBear              2.5
                                  AgentMunky            4.0
                                  AgentZero             2.0
                                  Ahoyhoyindeed         4.5
                                  Aiptasia              3.0
                                  AlCaponeJunio

Display the head of the pivot table, but only for 5 users (columns are users)

In [33]:
df_wide.ix[0:5, 0:5]

review_profilename                         0110x011  02maxima  03SVTCobra  \
               beer_name                                                    
review_overall #9                               NaN       NaN         NaN   
               120 Minute IPA                   NaN       NaN         NaN   
               1554 Enlightened Black Ale       NaN       NaN         NaN   
               60 Minute IPA                    NaN       NaN         NaN   
               90 Minute IPA                      5       NaN         NaN   

review_profilename                         05Harley  0Naught0  
               beer_name                                       
review_overall #9                               NaN       NaN  
               120 Minute IPA                     4       NaN  
               1554 Enlightened Black Ale       NaN       NaN  
               60 Minute IPA                    NaN       NaN  
               90 Minute IPA                      4       NaN

### Discussion: what do you notice in this table?

#### Data munging
Set Nans to zero

In [34]:
df_wide = df_wide.fillna(0)

Check that columns are users

In [35]:
df_wide.columns[:10]

Index([u'0110x011', u'02maxima', u'03SVTCobra', u'05Harley', u'0Naught0',
       u'0beerguy0', u'0runkp0s', u'0tt0', u'1000Bottles', u'1001111.0'],
      dtype='object', name=u'review_profilename')

Check that rows are beers

In [36]:
df_wide.index.levels[0]
beer_names = df_wide.index.levels[1]

### Calculate distance between beers

We're going to use cosine_similarity from scikit-learn to compute the distance between all beers

Imports

In [37]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances

Apply cosine similarity to df_wide to calculate pairwise distances

In [38]:
dists = cosine_similarity(df_wide)
dists

array([[ 1.        ,  0.27540494,  0.27410345, ...,  0.32928048,
         0.34805798,  0.31249922],
       [ 0.27540494,  1.        ,  0.25151873, ...,  0.2854835 ,
         0.23301356,  0.2802485 ],
       [ 0.27410345,  0.25151873,  1.        , ...,  0.31629515,
         0.22521858,  0.2737628 ],
       ..., 
       [ 0.32928048,  0.2854835 ,  0.31629515, ...,  1.        ,
         0.28025764,  0.34504013],
       [ 0.34805798,  0.23301356,  0.22521858, ...,  0.28025764,
         1.        ,  0.25526913],
       [ 0.31249922,  0.2802485 ,  0.2737628 , ...,  0.34504013,
         0.25526913,  1.        ]])

### Discussion: what type of object is dists?

Convert dists to a Pandas DataFrame, use the index as column index as well (distances are a square matrix).

In [39]:
dists = pd.DataFrame(dists)
dists.columns = beer_names
dists.index = beer_names
dists.ix[0:10, 0:10]

beer_name,#9,120 Minute IPA,1554 Enlightened Black Ale,60 Minute IPA,90 Minute IPA,Aecht Schlenkerla Rauchbier Märzen,AleSmith IPA,AleSmith Speedway Stout,Allagash White,Alpha King Pale Ale
beer_name,,,,,,,,,,
#9,1.000000,0.275405,0.274103,0.388364,0.365175,0.253841,0.228479,0.227612,0.340681,0.293315
120 Minute IPA,0.275405,1.000000,0.251519,0.378258,0.410366,0.262425,0.315971,0.337541,0.282273,0.336796
1554 Enlightened Black Ale,0.274103,0.251519,1.000000,0.319887,0.314028,0.252486,0.266866,0.261761,0.260275,0.307296
60 Minute IPA,0.388364,0.378258,0.319887,1.000000,0.533042,0.316928,0.312343,0.307627,0.360975,0.385249
90 Minute IPA,0.365175,0.410366,0.314028,0.533042,1.000000,0.312861,0.344218,0.358754,0.356804,0.418582
Aecht Schlenkerla Rauchbier Märzen,0.253841,0.262425,0.252486,0.316928,0.312861,1.000000,0.244490,0.246063,0.297672,0.263248
AleSmith IPA,0.228479,0.315971,0.266866,0.312343,0.344218,0.244490,1.000000,0.521889,0.277409,0.400741
AleSmith Speedway Stout,0.227612,0.337541,0.261761,0.307627,0.358754,0.246063,0.521889,1.000000,0.273930,0.420247
Allagash White,0.340681,0.282273,0.260275,0.360975,0.356804,0.297672,0.277409,0.273930,1.000000,0.295666


Select some beers and look their distances to other beers

In [40]:
beers_i_like = ['Sierra Nevada Pale Ale', '120 Minute IPA', 'Allagash White']
dists[beers_i_like].head()

beer_name,Sierra Nevada Pale Ale,120 Minute IPA,Allagash White
beer_name,,,
#9,0.373968,0.275405,0.340681
120 Minute IPA,0.301693,1.000000,0.282273
1554 Enlightened Black Ale,0.330033,0.251519,0.260275
60 Minute IPA,0.459641,0.378258,0.360975
90 Minute IPA,0.441189,0.410366,0.356804


Sum the distances of my favourite beers by row, to have one distance from each beer in the sample

In [43]:
beers_summed = dists[beers_i_like].apply(lambda row: np.sum(row), axis=1)
#beers_summed = np.sum(dists[beers_i_like], axis=1)

In [44]:
beers_summed

beer_name
#9                                                    0.990054
120 Minute IPA                                        1.583966
1554 Enlightened Black Ale                            0.841827
60 Minute IPA                                         1.198874
90 Minute IPA                                         1.208359
Aecht Schlenkerla Rauchbier Märzen                    0.853397
AleSmith IPA                                          0.887446
AleSmith Speedway Stout                               0.881959
Allagash White                                        1.634784
Alpha King Pale Ale                                   0.976927
Anchor Liberty Ale                                    1.064927
Anchor Old Foghorn                                    0.944878
Anchor Porter                                         0.999485
Anchor Steam Beer                                     1.101660
ApriHop                                               1.089122
Arrogant Bastard Ale                         

#### Performance

Optional: which one is faster? use ```%timeit``` to check

In [45]:
%timeit dists[beers_i_like].apply(lambda row: np.sum(row), axis=1)

100 loops, best of 3: 5.16 ms per loop


In [46]:
%timeit np.sum(dists[beers_i_like], axis=None)

The slowest run took 4.18 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 339 µs per loop


#### Ranking

Sort summed beers from best to worse

In [42]:
beers_summed = beers_summed.order(ascending=False)
beers_summed

beer_name
Sierra Nevada Pale Ale                        1.654205
Allagash White                                1.634784
120 Minute IPA                                1.583966
HopDevil Ale                                  1.224217
Sierra Nevada Celebration Ale                 1.215156
90 Minute IPA                                 1.208359
60 Minute IPA                                 1.198874
Stone Ruination IPA                           1.194210
Stone IPA (India Pale Ale)                    1.193193
Storm King Stout                              1.192405
Arrogant Bastard Ale                          1.189981
Sierra Nevada Bigfoot Barleywine Style Ale    1.178245
Prima Pils                                    1.178093
Brooklyn Black Chocolate Stout                1.156365
Ayinger Celebrator Doppelbock                 1.148356
Hennepin (Farmhouse Saison)                   1.147501
Samuel Adams Boston Lager                     1.146304
Hop Rod Rye                                   1.140271


Filter out the beers used as input and transform to list

In [27]:
ranked_beers = beers_summed.index[beers_summed.index.isin(beers_i_like)==False]
ranked_beers = ranked_beers.tolist()
ranked_beers[:5]

['HopDevil Ale',
 'Sierra Nevada Celebration Ale',
 '90 Minute IPA',
 '60 Minute IPA',
 'Stone Ruination IPA']

###Pair Programming!

Define a function that does what we just did for an arbitrary input list of beers. it should also receive the maximum number of beers requested n as optional parameter.

In [51]:
def beer_rec(beer_list, beer_num):
    beer_rec_sum = np.sum(dists[beer_list], axis=1)
    ranked_beers_rec = beer_rec_sum.order(ascending=False)
    ranked_beers_rec_drop = ranked_beers_rec.index[ranked_beers_rec.index.isin(beer_list)==False]
    print list(ranked_beers_rec_drop[:beer_num])

Test your function. Find the 10 beers most similar to "120 Minute IPA"

In [52]:
beer_rec(['HopDevil Ale','90 Minute IPA'], 10)

['Storm King Stout', '60 Minute IPA', 'Stone IPA (India Pale Ale)', 'Stone Ruination IPA', 'Hop Wallop', 'Sierra Nevada Celebration Ale', 'Prima Pils', 'Arrogant Bastard Ale', 'Sierra Nevada Bigfoot Barleywine Style Ale', 'Old Rasputin Russian Imperial Stout']


Try again with the 10 beers most similar to ["Coors Light", "Bud Light", "Amstel Light"]

In [53]:
beer_rec(["Coors Light", "Bud Light"],10)

['Miller Lite', 'Budweiser', 'Corona Extra', 'Samuel Adams Boston Lager', 'Heineken Lager Beer', 'Blue Moon Belgian White', 'Guinness Draught', 'Miller High Life', 'Samuel Adams Summer Ale', 'Sierra Nevada Pale Ale']


Optional: register an account on yhat and deploy your model following the instructions [here](https://docs.yhathq.com/python/examples/beer-recommender) and [here](http://nbviewer.ipython.org/gist/glamp/20a18d52c539b87de2af)

## Recsys

A python library for implementing a recommender system. If you'd like to, I recommend you explore this project. It's an efficient way to get a recommendation engine off the ground. The example below uses SVD.

In [1]:

##install python-recsys

### first install dependencies

! pip install csc-pysparse networkx divisi2

### then install recsys
! git clone https://github.com/python-recsys/python-recsys.git
# ! cd python-recsys/

! cd python-recsys/ && python setup.py install


fatal: destination path 'python-recsys' already exists and is not an empty directory.
running install
running bdist_egg
running egg_info
writing requirements to python_recsys.egg-info/requires.txt
writing python_recsys.egg-info/PKG-INFO
writing top-level names to python_recsys.egg-info/top_level.txt
writing dependency_links to python_recsys.egg-info/dependency_links.txt
reading manifest file 'python_recsys.egg-info/SOURCES.txt'
writing manifest file 'python_recsys.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.5-x86_64/egg
running install_lib
running build_py
creating build/bdist.macosx-10.5-x86_64/egg
creating build/bdist.macosx-10.5-x86_64/egg/recsys
copying build/lib/recsys/__init__.py -> build/bdist.macosx-10.5-x86_64/egg/recsys
creating build/bdist.macosx-10.5-x86_64/egg/recsys/algorithm
copying build/lib/recsys/algorithm/__init__.py -> build/bdist.macosx-10.5-x86_64/egg/recsys/algorithm
copying build/lib/recsys/algorithm/baseclass.py -> build/bdist.macosx-

In [2]:
! pip freeze

abstract-rendering==0.5.1
alabaster==0.7.3
appscript==1.0.1
argcomplete==0.8.9
astropy==1.0.3
Babel==1.3
backports.ssl-match-hostname==3.4.0.2
bcolz==0.9.0
beautifulsoup4==4.3.2
binstar==0.11.0
bitarray==0.8.1
blaze==0.8.0
blz==0.6.2
bokeh==0.9.0
boto==2.38.0
Bottleneck==1.0.0
cdecimal==2.3
certifi==14.5.14
cffi==1.1.0
clyent==0.3.4
colorama==0.3.3
conda==3.16.0
conda-build==1.14.1
conda-env==2.4.2
configobj==5.0.6
cryptography==0.9.1
csc-pysparse==1.1.1.4
csc-utils==0.6.7
Cython==0.22.1
cytoolz==0.7.3
datashape==0.4.5
decorator==3.4.2
Divisi2==2.2.5
docutils==0.12
enum34==1.0.4
fastcache==1.0.2
Flask==0.10.1
funcsigs==0.4
gevent==1.0.1
gevent-websocket==0.9.3
greenlet==0.4.7
grin==1.2.1
h5py==2.5.0
idna==2.0
ipaddress==1.0.7
ipython==3.2.0
itsdangerous==0.24
jdcal==1.0
jedi==0.8.1
Jinja2==2.7.3
jsonschema==2.4.0
llvmlite==0.5.0
lxml==3.4.4
MarkupSafe==0.23
matplotlib==1.4.3
mistune==0.5.1
mock==1.0.1
multipledispatch==0.4.7
networkx==1.9.1
nltk==3.0.3
nose==1.3.7
numba==0.19.1
numexpr

Load recsys.algotihm, set VERBOSE = True import SVD class

In [3]:
! pwd
import recsys.algorithm
recsys.algorithm.VERBOSE = True
from recsys.algorithm.factorize import SVD

/Users/nathanbackblaze/projects/GeneralAssembly/DAT_SF_16/labs


/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/divisi2/fileIO.py:2: UserWarning: Module recsys was already imported from /Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/python_recsys-0.2-py2.7.egg/recsys/__init__.pyc, but /Users/nathanbackblaze/projects/GeneralAssembly/DAT_SF_16/labs is being added to sys.path
  from pkg_resources import resource_filename


Let's look at the files

In [4]:
! ls movielens

README      movies.dat  ratings.dat users.dat


Import 'movies.dat' to a 'movies' pandas dataframe. Make sure you name the columns, use the correct separator and define the index.

In [7]:
movies = pd.read_table('movielens/movies.dat', sep='::', names= ['ITEMID', 'Title', 'Genres'], index_col= 'ITEMID')

/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:648: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  ParserWarning)


In [8]:
movies.head()

,Title,Genres
ITEMID,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy


Import 'ratings.dat' to a 'ratings' pandas dataframe. Make sure you name the columns, use the correct separator.

In [9]:
ratings = pd.read_table('movielens/ratings.dat', sep='::', names= ['UserID','MovieID','Rating','Timestamp'])

In [10]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


Initialize an SVD instance

In [11]:
svd = SVD()

Populate it with the data from the ratings dataset, using the built in load_data method

In [12]:
svd.load_data(filename='./movielens/ratings.dat', sep='::', format={'col':0, 'row':1, 'value':2, 'ids': int})

Loading ./movielens/ratings.dat
..........|


Compute SVD

$M=U \Sigma V^T$:

In [13]:
k = 100
svd.compute(k=k, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True)

Creating matrix (1000209 tuples)
Matrix density is: 4.4684%
Updating matrix: squish to at least 10 values
Computing svd k=100, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True
[WARNING] mean_center is True. svd.similar(...) might return nan's. If so, then do svd.compute(..., mean_center=False)


you can also save the output SVD model (in a zip file)

In [14]:
# svd.compute(k=k, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True, savefile='/tmp/movielens')

Reload a saved model:

In [15]:
# svd2 = SVD(filename='/tmp/movielens')

Find the ITEMID number for "Toy Story (1995)"

In [16]:
movies[movies.Title == "Toy Story (1995)"]

,Title,Genres
ITEMID,,
1,Toy Story (1995),Animation|Children's|Comedy


Find the ITEMID number for "Bug's Life, A (1998)"

In [17]:
movies[movies.Title == "Bug's Life, A (1998)"]

,Title,Genres
ITEMID,,
2355,"Bug's Life, A (1998)",Animation|Children's|Comedy


Compute similarity between the two movies

In [18]:
ITEMID1 = 1    # Toy Story (1995)
ITEMID2 = 2355 # A bug's life (1998)
print svd.similarity(ITEMID1, ITEMID2)
# print svd2.similarity(ITEMID1, ITEMID2) to check

0.677069366773


Get movies similar to Toy Story

In [19]:
svd.similar(ITEMID1)

[(1, 0.99999999999999978),
 (3114, 0.87060391051017305),
 (2355, 0.67706936677314977),
 (588, 0.58073514967544992),
 (595, 0.46031829709744226),
 (1907, 0.44589398718134982),
 (364, 0.42908159895577563),
 (2081, 0.42566581277822413),
 (3396, 0.42474056361934953),
 (2761, 0.40439361857576017)]

Predict rating for a given user and movie, $\hat{r}_{ui}$

In [20]:
MIN_RATING = 0.0
MAX_RATING = 5.0
ITEMID = 1
USERID = 1
svd.predict(ITEMID, USERID, MIN_RATING, MAX_RATING)

5.0

In [21]:
svd.get_matrix().value(ITEMID, USERID)

5.0

Recommend non rated movies to a user

In [22]:
svd.recommend(USERID, is_row=False)

[(2028, 5.4018452642332546),
 (527, 5.3498144196809516),
 (2905, 5.2133848204673132),
 (318, 5.2052108435955446),
 (1193, 5.1942189963876562),
 (3114, 5.1753939214583697),
 (1, 5.1714259073839521),
 (2019, 5.1037438278754719),
 (1178, 5.0962756861446641),
 (1207, 5.090305272922329)]

Which users should see Toy Story? (e.g. which users -that have not rated Toy Story- would give it a high rating?)

In [23]:
svd.recommend(ITEMID)

[(869, 6.8215500393190904),
 (4086, 6.2667649038936908),
 (549, 6.2394061595542869),
 (1343, 6.2163075783431427),
 (1586, 6.039893928886932),
 (840, 5.9616632765170472),
 (1676, 5.896233772781037),
 (4595, 5.88945710113423),
 (2691, 5.8735094161364714),
 (2665, 5.8498694241604259)]

Find out more here: [https://github.com/ocelma/python-recsys](https://github.com/ocelma/python-recsys)